#  Final Report
## Predicting overperformance of the S & P 500 by Super Bowl advertisers


## 1. Introduction
### a. Problem

The Super Bowl is the premier advertising event of the year with this year’s 30 second
ads selling for up to $7 million<sub>[3]</sub>. Marketing professors Chuck Tomkovick and Rama Yelkur,
University of Wisconsin-Eau Claire, found that on average advertisers outperform the S&P 500
by 1\% over the time period one week before until 1 week after the game. They also examined
longer term behavior in 2009 and 2010, finding that over 60\% of advertisers were outperforming
at midyear with only a slight drop off by year end<sub>[2]</sub>.


The initial purpose of this study will be to confirm the conclusions of Tomovick and Yelkur
for the midyear overperformance. Additionally, by analyzing the data for a longer
period, a further objective will be to predict the level of over/underperformance of a given
advertiser so that investors will be able to determine effective investment strategies using a
slate of winners and losers chosen from the current year’s advertisers. Since even the naive
approach of buying all the advertisers a week before the game would overperform the S&P 500
any refinement which could determine which stocks are likely winners as opposed to losers
could greatly increase the rate of return relative to market benchmarks.


The Kiplinger opinion piece presents an anecdotal theory on which stocks might
underperform based on time since IPO<sub>[1]</sub>. While certainly worth investigating, there is little hard
evidence presented in the article and the idea is heavily reliant on data for two specific years:
the dot com ads in 2000 and last year’s plethora of crypto ads. This might also be useful as a
feature in potential future work with predictions about bubbles based on the number of new
advertisers in the Super Bowl.

### b. Client 

  
The clients for this question would be a financial investment firms, advertising companies or the advertisers themselves. The financial companies would want a list of winners and losers and a trading strategy to maximize profit from the list. The ad companies would gain valuable marketing and sales information by knowing which companies have the greatest chance of recouping the costs of these expensive ads in their stock price. Finally, the companies themselves would like to know if these expensive ads are worthwhile.
    
    
### c. Binary classification

  
The question is set up as a Binary Classification problem with "1", representing overperforming  and "0", underperforming (technically not overperforming). The goal in these problems is to build a model that can make predictions about which class a new input data point belongs to, based on the patterns and features present in the prior data. 


There are several types of models to choose from for binary classification.

**Logistic Regression:** A simple and widely used model for binary classification. It models the probability of an instance belonging to a particular class using the logistic function. Unfortunately, this works well when the relationship between the features and the target variable is relatively simple which seems unlikely here. The primary advantage is speed which is less important in this case with a small number of instances.

**K-Nearest Neighbors (KNN):** A simple instance-based classification method that classifies an instance based on the majority class of its k nearest neighbors in the feature space. Simple, but can be slow for large datasets. Intuitively, for this problem, seems like it would use a lot of numeric features in the distance metric and may have difficulty with some of the categorical features which lack ordinality.

**Decision Trees:**  They make binary decisions at each node, leading to a leaf node representing a predicted class. Decision trees are easy to interpret but are typically used in ensemble methods.

**Random Forest:** An ensemble method that combines multiple decision trees to improve overall performance and reduce overfitting. Each tree is trained on a different subset of the data, and the final prediction is the majority vote (for classification) of the individual tree predictions.

**Gradient Boosting:** An ensemble of weak learners (typically decision trees) in a sequential manner. Each new tree corrects the errors made by the previous ones, resulting in a strong predictive model. 

**Support Vector Machines (SVM):** Find the hyperplane that best separates the two classes while maximizing the margin between them. Effective in cases where the classes are not linearly separable but are in a higher-dimensional space. Sort of a higher dimensional regression which may work well with the categorical features.

**Naive Bayes** and **Neural Networks** were left off the list due to the small size of the data.

## 2. Data

### a. Raw Sources
The data came from the following three sources and ended up needing extensive cleaning:

+ https://www.pro-football-reference.com/super-bowl/

        The data on this webpage was scraped to a text file that was saved as `df3` for cleaning.
+ https://www.kaggle.com/datasets/prondeau/superbowlads

        This is the main data set from Kaggle and was saved as `df4` for cleaning.
+ https://datahub.io/core/employment-us/datapackage.json

        This is some annual economic data that was saved as `usafacts` 

### b. Cleaning, Merging and Feature Extraction
Each of these dataframes required some manipulation.

* `df3` was not in bad shape. There are 11 columns:

`Date`,  `SB`, `Winner`, `Pts` , `Loser` , `Pts.1` ,  `MVP`  ,  `Stadium`  , `City`,  `State` , `Year`  

This data is mostly self-explanatory and was primarily used for the actual date of the Super Bowl each year. It may have been interesting to leave in the cities to see if a large population impacts the results, but this was not done at this time.

* `df4` The main data set was a mess. I took some manipulation of the csv file just to load but once it did the data was fairly simple, with only 4 columns:
`Year`, `Product Type`, `Product/Title` and  `Plot/Notes` , however it required extensive cleaning to extract additional features. Initially the data looks like this.



<img src="fig1.jpeg" width=600 height=400 />

The file has a listing of 563 Super Bowl commercials with the year, product type, product/commmercial title, and sometimes a plot.

We are mostly interested in the product type and getting the parent company from the product. 
+ **Year**  : has the year of the Super Bowl with only one minor problem to fix to have all entries correctly assigned
+ **Product Type** : the type of product advertised, initially has too many classes more details below
+ **Product/Title** :  this has the brand of the advertiser and the title of the commercial, will be split into 2 columns `Brand` and `Title`
+ **Plot/Notes** : a brief description of the ad, often missing and not of much seeming value for our purposes

Next, merge  `Date` from `df3` into `df4` nn the `Year` column choosing to leave off the other columns.

Now, the messy bit. `Product Type` starts with far to many categories for the size of the data set, so they were consolidated into the following: 

        'Car', 'Goods', 'Technology', 'Soft drink', 'Food', 'Alcohol','Service', 'Clothing', 
        'Film', 'Sports', 'Website', 'TV', 'Gaming', 'Wireless', 'Car / Film'

The process was essentially the same for each `Product_type` and runs the same as the following or `Film`. Use ChatGPT to get the studio that made each movie. The result was copied to Sublime Text keeping the original index from `df4` as `row` when loaded into the df `filmstudios`. 

Now, got the Ticker symbols by using ChatGPT again on `filmstudios` saved in Sublime Text as `filmticker` similarly maintaining the `row`. Do this to similarly get the ticker symbols for all the categories and merge them all back into a single dataframe, `tickers_added` which now looks like this:

<img src="fig2.jpeg" width=800 height=700 />

Notice that the names of original `df3` columns have been changed into more standard forms and that `Brand` and `Title` have been extracted from the `Brand_title` column by splitting on  "  , and cleaning up afterwards. 

Since `Date` is already in a datetime format is is easy to generate `Mkt_Date` and `Mkt_Date_plus6`, chosen to be the Monday after the Superbowl and 183 days following that date. This spacing guarantees that on `Mkt_Date_plus6` the market is open as this will be either a Monday or Tuesday. 

Next is to get closing prices for the dates and tickers. I used two methods in an effort to get as many as possible.
First, I used 

https://www.kaggle.com/datasets/paultimothymooney/stock-market-data/code?resource=download
Drop all the rows out that have no `Ticker` symbol.
Create a `load_stock` function, then load all the stocks in the list of tickers that are found into `stocks` and find the value in the data set above then merge back into `df_test` and try the second method. 
Take `tickers_added` and load this file in excel using the stocks datatype, convert the dates in excel and extract closing prices from `STOCKHISTORY` and from the general data for the stock extract `year_inc`, `market_cap_cur` and `shares_out` for the year of incorporation, current market cap, and current shares outstanding. Include the S & P 500 as `spx_close` and `spx_plus6`, and compute the return for both the stock and the S & P 500 as `rtn` and `spx` and the difference `rtn_vs_spx`.
The 16 missing `spx` values are missing at least one close of the S&P and are essentially the old ads which are the first 16 original rows. Data was sparse for the years anyway so drop these out.

The second method yields 333 prices out of 556 rows. Now just want to see if we caught some in our earlier attempt that were not captured here. What happens if we merge on rows?

Drop everything that is repeated or unneeded and plop into new dataframe `df` which has columns


<img src="fig3.jpeg" width=500 height=500 />

Define a function to calculate number of previous years advertised, `Previous_SBs` and a function to output the number of ads in a year, `YearlyAdCount`.
Create a new column `New` based on whether the company incorporated within 8 years of the ad airdate and clean up column names.
Now go back again and compute the return for the stock and S & P 500 for the 183 day period for each row. Define the target column `overperform` to be '1' when `return` is greater than `SP_return` and '0' otherwise.



* `usafacts` :  add in some generic annual economic data for the year prior to the Super Bowl

The columns `Ave_inflation_rate`, `CPI`, `USD_per_euro`, `Annual_change_GDP`, `VIX`, and  `change_in_businesses` are added in reflecting  the average inflation rate , consumer price index , change in annual GDP, the VIX or 'disater index' and the change in the number of businesses.

There are some extra columns at this point. 

The dataset still needs some cleanup. `row` doesn't hold any information anymore, so we can drop that.

Let's look at other variables, `Year` looks fine all values filled in. 

`Product_type` was worked on hard and they all seem to have value so let's keep them.

The `Brand` and `Title` features don't really seem to differentiate much, since they have so many unique values. It is hard to see what either adds to `Ticker` so they are dropped.

`Mkt_Date` and  `Mkt_Date_plus6` were derived from the year then used to extract other data so they should be dropped as well.

`close` and `close_plus6` are both obviously linked to `return` and if all the instances without a return are dropped both should automatically have no Na values. Leave them for now.

`market_cap_cur` is the current(not historical) market cap of the stock. 

Drop all the rows without a return. This leaves 388 rows with two of them not having a value for `SP_return`. Thes are both old so drop them.

Drop all the rows that are for multiple ads by the same ticker in the sm, USD_per_euroe year leaving one for each `Ticker` and `Year`. This now leaves 232 unique advertisers and years. There are few remaining null values in `market_cap_cur`, `shares_out`,`VIX` and `change_in_businesses` fill them each with the mean and `USD_to_Euro`with "1". 

Features `close`, `close_plus6`, `spx_close`, `spx_plus6`, `return` and `SP_return` contain future information should be deleted from the model.

This data frame is saved as `df_clean` and now has shape (232, 16) with no missing values.

### c. **EDA**
Let's look at the correlation heatmap and go through the numerical features first.

<img src="figures/fig4.jpeg" width=800 height=800 />

* **Year:** The year is both an identifier or subindex and a feature there are several features with correlations to the year which is not surprising since many of the extracted features depend on the year such as annual economic data or the derived features `Previous_SB` and `New`. Interestingly, it also seems that the `Yearly_Ad_Count` goes up in time as well. 

<img src="figures/fig6.jpg" width=800 height=800 />


As seen in the histogram the two years 2009 and 2010 were particularly good years with a 62.5\% of the advertisers beating the S & P in those years even though the average for all years is around 43\%.
This answers one of the initial questions as it appear the level of overperformance seen in 2009 and 2010 was unusually high. There appear to be some gaps still in the data in addition to a few really bad years for overperformance. It is also worth noting that the years with highest percentage of overperformance are mostly years with a low number of advertisers. 

The next group of features all came out of Excel's `STOCK` datatype

* **market_cap_cur:**  The current market cap of the `Ticker` this does not depend on the year but is the current value from Microsoft Excel. There does not seem to be any strong relationship here except with `Year`, `CPI` and `shares_out` but it seems to have some value in that while the median value is about the same for over/underperformers, the mean is nearly twice as large for underperformers.

* **shares_out:** The number of shares outstanding which with `market_cap_cur` will give the current stock price. Like that feature, this one seems to have fairly weak relationships with most of the other features and the target. 

* **year_inc** The year incorporated give an idea of how firmly established a company is. This required cleaning for some of the older companies which merged. This feature has correlations to the derived features `New`, `Previous_SB`, and `Yearly_Ad_Count`.

This next group were all derived from the original data `df3`

* **Previous_SBs:** This is the number of previous times the company has been a Super Bowl advertiser. While not very correlated to the target this has interesting correlations to the economic statistics.

* **Yearly_Ad_Count:** This is a count of how many ads the company had in that Super Bowl it will usually be 1 meaning the company's only  put out this ad that year.

* **New:** This is a binary column, 1 if the company is less than 8 years since incorporation 0 otherwise. This also has some interesting correlations to the economic statistics.

Finally there are the general economic features.

* **Ave_inflation_rate:** This is the previous year's average inflation rate.

* **CPI:** This is the previous years's average Consumer Price Index another measure of inflation.'

* **USD_per_Euro:** The exchange rate Dollars per Euro averaged over the previous year.

* **Annual_change_GDP:** This is the change in annual Gross Domestic Product reported at the end of the previous year. This is an indicator of economic growth/decline.

* **VIX:** Known as the "fear index" this is a measure of a basket of future options on the S & P 500 and is typically negatively correlated to the S & P 500.

* **change_in_businesses:** This measures the net change in the number of businesses the previous year, the number of new businesses minus the number of business closings.

This just leaves the two categorical features `Ticker` which is also an identifier in the index and `Product_type` 

* **Product_type:** Looking at the bar graphs below, there is large variability in the mean for each group and ignoring the undersampled group `Gaming` there are five groups `Food` , `Alcohol`, `Service`, `Soft Drink` and `TV` which had on average a better than 50\% chance of overperforming. `Websites` and `Cars` perform the worst at under 30\%, with `Clothing`, `Technology` and `Wireless` not much better.


<table><tr>
<td> <img src="figures/fig7.jpg" width=400 height=200 /> </td>
<td> <img src="figures/fig8.jpg" width=400 height=200 /> </td>
</tr></table>

 
When looking at `Product_type` along with `Previous_SBs` some interesting patterns emerge. 
- Having a previous ad helps for `TV` but hurts for `Film`
- Previous ads help `Alcohol` enormously going from 54\% to 67\%, but does not seem to have the same effect on the seemingly similar `Soft Drink`
- `Services`, `Clothing`  and `Technology`, however, do appear to have the same reputational? benefit as `Alcohol`
- `Food` and `Goods` vary seemingly due to small samples as do some of the others
- Only the big categories `Alcohol`, `Soft Drink`, `Food, `Car` and `Film` had an advertiser with more than 5 previous ad years

Similarly, looking at `Product_type` along with `Yearly_Ad_Count` generates some interesting patterns. 
- Having only a single ad seems to help `Film`, `Food` and `Service` 
- `Soft drink`, `TV` and  `Alcohol`  are the opposite and benefit from multiple ads as does `Car` to a small degree



* **Ticker:** This is the stock symbol for the parent company. This along with the year identify a unique record.


## 3. Model
### a. Preprocessing
Much of the preprocessing was already completed as there are no null values left. The main thing is to split the data into the training (80\%) and test(20\%) and apply the numerical and categorical transformers. These are chosen to be `StandardScaler` for numerical columns and `OneHotEncoding` for the two categorical columns `Ticker` and `Product_type`. 

Next I fit a Pipeline on the training set with Logistic Regression as the regressor. The AOC-ROC was .583 so tat was a good start with just default values.
Knowing that it was ready to go we try out the models from the introduction.

### Finding the Best Model
####Logistic Regression 
First, I tried Logistic Regression with various values of `C`. As seen in the results the best precision is around 60\% and the best recall is 35\%. Not really very good. 


<img src="figures/fig9.jpeg" width=400 height=200 />

#### Cross-validation Results
All five models were then tested with 4-fold cross-validation with the following results, all the scores are the mean under cross-validation.

<img src="figures/fig11.jpeg" width=800 height=600 />



Based on these intitial results GridSearch was run Random Forest,Gradient Boosting and KNN  to fine tune each looking for improvements to choose the best model. The reslults are in the table.


<img src="figures/fig12.jpg" width=400 height=400 />


Based on these results the Random Forest Classifier was chosen with the optimum hyperparameters `min_samples_leaf=4` and  `min_samples_split=5`.
When this is run on the test set with cross validation we get Accuracy =  0.53,  Recall =  0.25 and ROC-AUC = 0.60. Which seems pretty good. The feature significance is shown below.


<img src="figures/fig13.jpg" width=600 height=400 />



The model is now run on the full dataset. Giving the following results.

Classification Report

<img src="figures/fig14.jpeg" width=400 height=400 />

Precision Recall Curve

<img src="figures/fig15.jpg" width=600 height=400 />

ROC Curve

<img src="figures/fig16.jpg" width=600 height=400 />



The model seems to work quite well. 
*The average return of the model is 16.20\% while average return of the S & P those years is 2.05\% and the average return of the Pick All strategy those years is 6.04\%. So the model seems to pick a good basket of stocks.
* The model beat the S & P 500 all  17  years where it made picks.
* The model beat picking all the stocks by 5 percentage points  12  times
* The model beat the S & P 500 by 5 percentage points  16  times.
* The model did not predict any overperformers of the years 1984, 1990, 1993, 1996, 1997, 1998, 1999, 2000, 2004, 2005, 2006, or 2020.

The comparison of performance is seen below.
<img src="figures/fig17.jpg" width=600 height=400 />

So while not everything turned out as anticipated the model seem to create a highly profitable portfolio which was the ultimate goal.

Recommendations would be to use this model for a potential investment portfolio for investor clients.
For advertising company clients the recommendation would be to focus sales some of the better categories like `Alcohol`, `Soft Drink`, `Service and `Food`.
The recommendations to the advertisers themselves would depend on which category they were in but would be to encourage companies in the 'winning' categories to continue advertising and depending on the group whether multiple ads are beneficial, while potentially discouraging ad purchaces from the 'losing' categories.


